In [ ]:
drive_folder = "Machine_Unlearning_Drive/NLPResults/"

ssd_folder = "SSD/"

scrub_folder = "SCRUB/"

github_folder = "Machine_Unlearning/"

!pip install datasets transformers scikit-learn torch torchvision seaborn matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import datasets
from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import copy

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

from Machine_Unlearning.Metrics.metrics import *

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

def seed_everything(seed):
  RNG = torch.Generator().manual_seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)
  np.random.seed(seed)
  return RNG

SEED = 44
SPLIT = 0.15
STARTING_ALPHA = 0.5 #Fixed starting alpha for NLP 
RNG = seed_everything(SEED)
results = {}

In [ ]:
go_emotions = load_dataset("go_emotions")
data = go_emotions.data

train = data["train"].to_pandas()
valid = data["validation"].to_pandas()
test = data["test"].to_pandas()

print(train.shape, valid.shape, test.shape)
# (43410, 3) (5426, 3) (5427, 3)

train.head()

In [ ]:
n_labels = 28
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in range(len(df)):
        temp = [0] * n_labels
        label_indices = df.iloc[i]["labels"]
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)

    return pd.DataFrame(one_hot_encoding)

train_ohe_labels = one_hot_encoder(train)
valid_ohe_labels = one_hot_encoder(valid)
test_ohe_labels = one_hot_encoder(test)

print(train_ohe_labels.shape)
#(43410, 28)

train = pd.concat([train, train_ohe_labels], axis=1)
valid = pd.concat([valid, valid_ohe_labels], axis=1)
test = pd.concat([test, test_ohe_labels], axis=1)

train.head()

In [ ]:
class GoEmotionDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class GoEmotionClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoEmotionClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example:
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = nn.BCEWithLogitsLoss()(outputs, targets.float())
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    return train_loss


def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example:
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = nn.BCEWithLogitsLoss()(outputs, targets.float())
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
bert_model = transformers.RobertaModel.from_pretrained("roberta-base")

def build_dataset(tokenizer_max_len):
    train_dataset = GoEmotionDataset(train.text.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoEmotionDataset(valid.text.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    test_dataset = GoEmotionDataset(test.text.tolist(), test[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    return train_dataset, valid_dataset, test_dataset

def build_dataloader(train_dataset, valid_dataset, test_dataset,  batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader, test_data_loader

def ret_model(n_train_steps, do_prob):
  model = GoEmotionClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

In [ ]:
config = {"tokenizer_max_len" : 40,
          "batch_size": 128,
          "dropout": 0.5,
          "epochs":0}

train_dataset, valid_dataset, test_dataset = build_dataset(config['tokenizer_max_len'])
train_data_loader, valid_data_loader, test_data_loader = build_dataloader(train_dataset, valid_dataset, test_dataset, config['batch_size'])
print("Length of Train Dataloader: ", len(train_data_loader))
print("Length of Valid Dataloader: ", len(valid_data_loader))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_train_steps = int(len(train_dataset) / config['batch_size'] * 10)


weights_pretrained = torch.load(github_folder +'best_model_goemotions.pt', map_location=device)

# load model with pre-trained weights
model = ret_model(n_train_steps, config['dropout'])
model.load_state_dict(weights_pretrained)

optimizer = AdamW(model.parameters(), lr=5e-05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=config['epochs'])
model.to(device)
#model = nn.DataParallel(model) #READ bout that



In [ ]:
def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    with torch.no_grad():
            for bi, d in enumerate(loader):
                ids = d["ids"]
                mask = d["mask"]
                targets = d["labels"]

                ids = ids.to(device, dtype=torch.long)
                mask = mask.to(device, dtype=torch.long)
                targets = targets.to(device, dtype=torch.float)


                outputs = net(ids=ids, mask=mask)

                correct_vec = (torch.argmax(targets,dim=1) == torch.argmax(outputs,dim=1))
                correct += correct_vec.sum().item()
                total += int(targets.size()[0])

    return correct / total

In [ ]:
def compute_entropies(net, loader):
    """Auxiliary function to compute per-sample losses"""

    #data_loader = torch.utils.data.DataLoader(loader.dataset, batch_size=1, shuffle=False, num_workers = 2, prefetch_factor = 10)
    prob = []
    DEVICE = next(net.parameters()).device
    with torch.no_grad():
        for d in loader:
            #batch = [tensor.to(next(model.parameters()).device) for tensor in batch]
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)


            outputs = net(ids=ids, mask=mask)
            prob.append(torch.nn.functional.softmax(outputs, dim=-1).data)

    p = torch.cat(prob)
    return (-torch.where(p > 0, p * p.log(), p.new([0.0])).sum(dim=-1, keepdim=False)).numpy(force=True)

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    DEVICE = next(net.parameters()).device
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    all_losses = []

    for d in loader:
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)


        logits = net(ids=ids, mask=mask)
        losses = criterion(logits, targets).sum(dim=1).numpy(force=True)
        #print(losses)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

In [ ]:
def readout(model,name):
  RNG = seed_everything(SEED)
  test_entropies = compute_entropies(model, test_data_loader)
  retain_entropies = compute_entropies(model, retain_loader)
  forget_entropies = compute_entropies(model, forget_loader)


  results[f"test_entropies_{name}"] = test_entropies.tolist()
  results[f"retain_entropies_{name}"] = retain_entropies.tolist()
  results[f"forget_entropies_{name}"] = forget_entropies.tolist()

  test_losses = compute_losses(model, test_data_loader)
  retain_losses = compute_losses(model, retain_loader)
  forget_losses = compute_losses(model, forget_loader)

  results[f"test_losses_{name}"] = test_losses.tolist()
  results[f"retain_losses_{name}"] = retain_losses.tolist()
  results[f"forget_losses_{name}"] = forget_losses.tolist()

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  gen = np.random.default_rng(1)
  if len(test_losses) > len(forget_losses):
    gen.shuffle(test_losses)
    test_losses = test_losses[: len(forget_losses)]
  else:
    gen.shuffle(forget_losses)
    forget_losses = forget_losses[: len(test_losses)]
    # make sure we have a balanced dataset for the MIA
  assert len(test_losses) == len(forget_losses)

  samples_mia = np.concatenate((test_losses, forget_losses)).reshape((-1, 1))
  labels_mia = [0] * len(test_losses) + [1] * len(forget_losses)

  mia_scores = simple_mia(samples_mia, labels_mia)

  print(
      f"The MIA has an accuracy of {mia_scores.mean():.3f} on forgotten vs unseen images"
  )

  results[f"MIA_losses_{name}"] = mia_scores.mean()

  gen = np.random.default_rng(1)
  if len(test_entropies) > len(forget_entropies):
    gen.shuffle(test_entropies)
    test_entropies = test_entropies[: len(forget_entropies)]
  else:
    gen.shuffle(forget_entropies)
    forget_entropies = forget_entropies[: len(test_entropies)]
    # make sure we have a balanced dataset for the MIA
  assert len(test_entropies) == len(forget_entropies)

  samples_mia = np.concatenate((test_entropies, forget_entropies)).reshape((-1, 1))
  labels_mia = [0] * len(test_entropies) + [1] * len(forget_entropies)

  mia_scores = simple_mia(samples_mia, labels_mia)

  print(
      f"The MIA has an accuracy of {mia_scores.mean():.3f} on forgotten vs unseen images"
  )

  results[f"MIA_entropies_{name}"] = mia_scores.mean()

  results[f"train_accuracy_{name}"] = accuracy(model, retain_loader)
  results[f"test_accuracy_{name}"] = accuracy(model, test_data_loader)
  results[f"forget_accuracy_{name}"] = accuracy(model, forget_loader)

  print("Train acc:"+ str(results[f"train_accuracy_{name}"]))
  print("Test acc:"+ str(results[f"test_accuracy_{name}"]))
  print("Forget acc:" +str(results[f"forget_accuracy_{name}"]))

In [ ]:
GEN1 = torch.Generator().manual_seed(42)
retain_set, forget_set = torch.utils.data.random_split(train_dataset,[1-SPLIT,SPLIT],GEN1)

forget_loader = torch.utils.data.DataLoader(
    forget_set, batch_size=128, shuffle=True, num_workers=2 , generator=RNG
)
retain_loader = torch.utils.data.DataLoader(
    retain_set, batch_size=128, shuffle=True, num_workers=2, generator=RNG
)

In [ ]:
readout(model,"original")

In [ ]:
def unlearning(net, retain, forget, validation, start_alpha = 0.1, alpha_sched = lambda start_a,a,max_ep,ep: a-(start_a/max_ep), lr = 0.001, forget_epochs = 27, use_scheduler = True):
    import math


    ### FORGETTING ###
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    optimizer = optim.AdamW(net.parameters(), lr= lr )
    forget_epochs = forget_epochs
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=forget_epochs)

    net.eval()

    alpha = start_alpha

    net.train()

    retain_iter = iter(retain)

    def entropy(outputs):

        p = torch.nn.functional.softmax(outputs, dim=-1)
        return (-torch.where(p > 0, p * p.log(), p.new([0.0])).sum(dim=-1, keepdim=False))

    for i in range(forget_epochs):

      net.eval()



      if i%5==0 :
        print("Computing current moments on test set")
        #val_loss, first_test_moment, second_test_moment, test_std = compute_moments(net, validation)
        val_loss = 0
        total_samples = 0
        for d in validation:
          ids = d["ids"]
          mask = d["mask"]
          targets = d["labels"]

          ids = ids.to(device, dtype=torch.long)
          mask = mask.to(device, dtype=torch.long)
          targets = targets.to(device, dtype=torch.float)


          logits = net(ids=ids, mask=mask)
          losses = criterion(logits, targets).mean(dim=1)

          total_samples += targets.size(dim=0)
          val_loss += torch.sum(losses).item()


        val_loss = val_loss / total_samples
        #train_mean, first_train_moment, second_train_moment,train_std = compute_moments(net,retain_loader)
        print("Computed moments: "+str(val_loss))


      ft_forget_losses = compute_losses(net, forget)
      ft_test_losses = compute_losses(net, test_data_loader)

      gen = np.random.default_rng(1)
      gen.shuffle(ft_test_losses)

      if len(ft_test_losses) > len(ft_forget_losses):
        ft_test_losses = ft_test_losses[: len(ft_forget_losses)]
      else:
        ft_forget_losses = ft_forget_losses[: len(ft_test_losses)]

      # make sure we have a balanced dataset for the MIA
      assert len(ft_test_losses) == len(ft_forget_losses)

      ft_samples_mia = np.concatenate((ft_test_losses, ft_forget_losses)).reshape((-1, 1))
      labels_mia = [0] * len(ft_test_losses) + [1] * len(ft_forget_losses)

      ft_mia_scores = simple_mia(ft_samples_mia, labels_mia)

      print(
          f"The MIA has an accuracy of {ft_mia_scores.mean():.3f} on forgotten vs unseen images"
      )
      mia_metric_scores.append(ft_mia_scores.mean())

      acc = 100.0 * accuracy(net, test_data_loader)
      print(f"Accuracy on test set: {acc:.1f} ")
      accuracy_metric_scores.append(acc)


      net.train()

      print("Forgetting epoch "+str(i))

      if i % (len(retain)//len(forget))==0:
        print("Resetting retain iterator...")
        retain_iter = iter(retain)


      print("using alpha: "+str(alpha))
      for c, d_f in enumerate(forget):
        ids_f = d_f["ids"]
        mask_f = d_f["mask"]
        targets_f = d_f["labels"]

        ids_f = ids_f.to(device, dtype=torch.long)
        mask_f = mask_f.to(device, dtype=torch.long)
        targets_f = targets_f.to(device, dtype=torch.float)

        net.zero_grad()
        out_f = net(ids=ids_f, mask=mask_f)

        d_r = next(retain_iter)

        ids_r = d_r["ids"]
        mask_r = d_r["mask"]
        targets_r = d_r["labels"]

        ids_r = ids_r.to(device, dtype=torch.long)
        mask_r = mask_r.to(device, dtype=torch.long)
        targets_r = targets_r.to(device, dtype=torch.float)

        out_r = net(ids=ids_r, mask=mask_r)


        forget_losses = criterion(out_f, targets_f).mean(dim=1)
        retain_losses = criterion(out_r,targets_r).mean(dim=1)

        #Forget loss metrics
        forget_mean = torch.mean(forget_losses)
        #forget_var = torch.mean((forget_losses-forget_mean)**2)
        #forget_std = forget_var**0.5
        #forget_skew = torch.mean((forget_losses-forget_mean)**3) / (forget_std**3)

        delta_val_loss =  (val_loss - forget_mean)
        #delta_first_moment = (first_test_moment - forget_var)
        #delta_second_moment = (second_test_moment - forget_skew)

        #Retain loss metric
        retain_mean = torch.mean(retain_losses)


        if c % 40 == 0:
          print("delta_val_loss: "+str(delta_val_loss.item()))
         # print("delta_first_moment: "+str(delta_first_moment.item()))
         # print("delta_second_moment: "+str(delta_second_moment.item()))

        #loss =  alpha*(torch.abs(delta_val_loss)+torch.abs(delta_first_moment)+torch.abs(delta_second_moment)) + (1-alpha)* retain_mean

        #loss =  alpha*(torch.nn.functional.relu(delta_val_loss)+torch.nn.functional.relu(delta_first_moment)) + (1-alpha)* retain_mean

        #loss =  alpha*(torch.abs(delta_val_loss)) + (1-alpha)* retain_mean
        loss =  alpha*(torch.nn.functional.relu(delta_val_loss**2)) + (1-alpha)* retain_mean


        loss.backward()
        optimizer.step()

      alpha = alpha_sched(start_alpha,alpha,forget_epochs,i)
      if use_scheduler:
        scheduler.step()


    net.eval()
    ft_forget_losses = compute_losses(net, forget)
    ft_test_losses = compute_losses(net, test_data_loader)

    gen = np.random.default_rng(1)
    gen.shuffle(ft_test_losses)
    if len(ft_test_losses) > len(ft_forget_losses):
      ft_test_losses = ft_test_losses[: len(ft_forget_losses)]
    else:
      ft_forget_losses = ft_forget_losses[: len(ft_test_losses)]
    # make sure we have a balanced dataset for the MIA
    assert len(ft_test_losses) == len(ft_forget_losses)

    ft_samples_mia = np.concatenate((ft_test_losses, ft_forget_losses)).reshape((-1, 1))
    labels_mia = [0] * len(ft_test_losses) + [1] * len(ft_forget_losses)

    ft_mia_scores = simple_mia(ft_samples_mia, labels_mia)

    print(
        f"The MIA has an accuracy of {ft_mia_scores.mean():.3f} on forgotten vs unseen images"
    )

    mia_metric_scores.append(ft_mia_scores.mean())

    acc = 100.0 * accuracy(net, test_data_loader)
    print(f"Accuracy on test set: {acc:.1f} ")
    accuracy_metric_scores.append(acc)

    net.eval()
    return net

In [ ]:
# Execute the unlearing routine. This might take a few minutes.
# If run on colab, be sure to be running it on  an instance with GPUs
accuracy_metric_scores = []
mia_metric_scores = []
delta_forget_losses = []
delta_val_losses = []
reg_factors = []
print(len(test_data_loader))
print(len(retain_loader))
print(len(forget_loader))
print(len(valid_data_loader))

def alpha_sched(start_a,a,max_ep,ep):
  #return start_a
  #if ep > 10:
    #return a - (start_a/(max_ep-10))
  #else:
   # return a
  #return start_a /(ep+1)
  return a - (start_a/(max_ep))

#TODO: Relu with other parameters
#TODO: Relu with entropy
weights_pretrained = torch.load(github_folder +'best_model_goemotions.pt', map_location=device)

# load model with pre-trained weights
ft_model = ret_model(n_train_steps, 0.0)
ft_model.load_state_dict(weights_pretrained)
ft_model.to(device)

#ft_model = copy.deepcopy(ft_model)
forget_epochs = int((len(retain_loader) / (len(forget_loader)*2)))
print(forget_epochs)

ft_model = unlearning(ft_model, retain_loader, forget_loader, valid_data_loader,start_alpha=STARTING_ALPHA, alpha_sched=alpha_sched, lr = 0.0003, forget_epochs = forget_epochs, use_scheduler = False)


In [ ]:
readout(ft_model, "ours")

In [ ]:
# Execute the unlearing routine. This might take a few minutes.
# If run on colab, be sure to be running it on  an instance with GPUs
accuracy_metric_scores = []
mia_metric_scores = []
delta_forget_losses = []
delta_val_losses = []
reg_factors = []
print(len(train_data_loader))
print(len(retain_loader))
print(len(forget_loader))

def alpha_sched(start_a,a,max_ep,ep):
  #return start_a
  #if ep > 10:
    #return a - (start_a/(max_ep-10))
  #else:
   # return a
  #return start_a /(ep+1)
  return a - (start_a/(max_ep))

#TODO: Relu with other parameters
#TODO: Relu with entropy
weights_pretrained = torch.load(github_folder +'best_model_goemotions.pt', map_location=device)

# load model with pre-trained weights
finetune_model = ret_model(n_train_steps, 0.0)
finetune_model.load_state_dict(weights_pretrained)
finetune_model.to(device)

#ft_model = copy.deepcopy(ft_model)

finetune_model = unlearning(finetune_model, retain_loader, forget_loader, valid_data_loader,start_alpha=0.0, alpha_sched=alpha_sched, lr = 0.0003, forget_epochs = forget_epochs*2, use_scheduler = False)

readout(finetune_model,"finetune")

In [ ]:
with open(drive_folder+f"results_NLP_SPLIT_{int(SPLIT*100)}%_SEED_{SEED}.json", 'w') as fout:
  json.dump(results, fout)